# Bài 4: Scraping Github cho các loại tập tin đặc biệt
*Cần để thực hiện yêu cầu 5*

In [1]:
# Import necessary libraries
import os
from github import Github
import base64

In [2]:
# Define user credentials
username = "keandk"
password = "..."

g = Github(username, password)

n = 5 # number of repo to download
i = 0 # start index

In [7]:
target_dir = "Powershell"
os.makedirs(target_dir, exist_ok=True)

In [8]:
# Search for repos written with a certain language
repos = g.search_repositories("language:powershell")

In [14]:
excluded_repos = ["winutil", "Scoop", "core"] # debug list

In [15]:
for repo in repos:
    repo_name = repo.name
    if repo_name in excluded_repos:
      continue
    target_dir_of_repo = os.path.join(target_dir, repo_name)
    print(repo_name)

    try:
        os.makedirs(target_dir_of_repo, exist_ok=True)
        i += 1
        contents = repo.get_contents("")

        while contents:
            file_content = contents.pop(0)

            if file_content.type == "dir":
                contents.extend(repo.get_contents(file_content.path))
            else:
                file_path = file_content.path
                filename = os.path.basename(file_path)
                extension = filename.split('.')[-1]

                if extension == "ps1":
                    file_data = base64.b64decode(file_content.content)
                    output_path = os.path.join(target_dir_of_repo, filename)

                    with open(output_path, "wb") as file_out:
                        file_out.write(file_data)

    except Exception as e:
        print(f"Error with repo {repo_name}: {str(e)}")

    if i == n:
        break

Windows10Debloater
Win11Debloat
SpotX


KeyboardInterrupt: 

# Bài 5: Phân loại tập tin theo kiểu

In [24]:
import os
from sklearn.feature_extraction.text import HashingVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline

javascript_path = "Java"
python_path = "Python"
powershell_path = "Powershell"

In [46]:
root, subdirs, files = next(os.walk(powershell_path))
for subdir in subdirs:
  for file in os.listdir(os.path.join(powershell_path, subdir)):
    try:
      os.rename(os.path.join(root + '\\'+ subdir, file), os.path.join(root, file))
    except FileExistsError:
      pass

In [47]:
corpus = []
labels = []
file_types_and_labels = [(javascript_path, -1), (python_path, 0), (powershell_path, 1)]
for files_path, label in file_types_and_labels:
  files = os.listdir(files_path)
  for file in files:
    file_path = files_path + "/" + file
    try:
      with open(file_path, "r") as myfile:
        data = myfile.read().replace("\n", "")
        data = str(data)
        corpus.append(data)
        labels.append(label)
    except:
      pass

In [49]:
X_train, X_test, y_train, y_test = train_test_split(corpus, labels, test_size=0.2, random_state=42)
text_clf = Pipeline(
  [
    ("vect", HashingVectorizer(input="content", ngram_range=(1, 3))),
    ("tfidf", TfidfTransformer(use_idf=True)),
    ("rf", RandomForestClassifier(class_weight="balanced")),
  ]
)

In [50]:
text_clf.fit(X_train, y_train)
y_pred = text_clf.predict(X_test)
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Confusion Matrix: ", confusion_matrix(y_test, y_pred))

Accuracy:  0.985663082437276
Confusion Matrix:  [[227   1   0]
 [  0  25   0]
 [  3   0  23]]


# Bài 6: Đo lường sự giống nhau giữa hai chuỗi

In [2]:
!pip install ssdeep

  Using cached ssdeep-3.4.tar.gz (110 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached cffi-1.17.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
Using cached cffi-1.17.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (467 kB)
Using cached pycparser-2.22-py3-none-any.whl (117 kB)
  Created wheel for ssdeep: filename=ssdeep-3.4-cp311-cp311-linux_x86_64.whl size=34045 sha256=42c7396de786ddcce0b1704b7fb62f00050ede103a81560ce0eb73521efa1629
  Stored in directory: /home/keanlt/.cache/pip/wheels/c0/cd/07/46fc341349658b05166e8e1f3eaadb44e90cd690aaa30ace7d
Successfully built ssdeep


In [3]:
import ssdeep
str1 = "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua."
str2 = "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore Magna aliqua."
str3 = "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore aliqua."
str4 = "Something completely different from the other strings."

In [4]:
hash1 = ssdeep.hash(str1)
hash2 = ssdeep.hash(str2)
hash3 = ssdeep.hash(str3)
hash4 = ssdeep.hash(str4)

print(hash1)
print(hash2)
print(hash3)
print(hash4)

3:f4oo8MRwRJFGW1gC6uWv6MQ2MFSl+JuBF8BSnJi:f4kPvtHMCMubyFtQ
3:f4oo8MRwRJFGW1gC6uWv6MQ2MFSl+JuBF8BS+EFECJi:f4kPvtHMCMubyFIsJQ
3:f4oo8MRwRJFGW1gC6uWv6MQ2MFSl+JuBF8BS6:f4kPvtHMCMubyF0
3:60QKZ+4CDTfDaRFKYLVL:ywKDC2mVL


In [5]:
print(ssdeep.compare(hash1, hash1))
print(ssdeep.compare(hash1, hash2))
print(ssdeep.compare(hash1, hash3))
print(ssdeep.compare(hash1, hash4))

100
39
37
0


![test](img/compare-exe-hexdump.png)

In [6]:
hash1 = ssdeep.hash_from_file("python-3.10.0-amd64.exe")
hash2 = ssdeep.hash_from_file("python-3.10.0-amd64-fake.exe")
ssdeep.compare(hash1, hash2)

100

# Bài 7: Trích xuất n-gram và chọn n-gram tốt nhất

In [1]:
import collections
from nltk import ngrams

In [2]:
file_to_analyze = "python-3.10.0-amd64.exe"

In [3]:
from typing import Any


def read_file(filepath: str) -> Any:
  with open(filepath, "rb") as myfile:
    data = myfile.read()
  return data

In [4]:
def byte_sequence_to_ngrams(byte_sequence, n):
  Ngrams = ngrams(byte_sequence, n)
  return list(Ngrams)

In [5]:
def binary_file_to_ngram_counts(file, n):
  filebyte_seq = read_file(file)
  file_ngram = byte_sequence_to_ngrams(filebyte_seq, n)
  return collections.Counter(file_ngram)

In [6]:
extract_ngrams = binary_file_to_ngram_counts(file_to_analyze, n=4)
print(extract_ngrams.most_common(10))

[((0, 0, 0, 0), 24290), ((139, 240, 133, 246), 1920), ((32, 116, 111, 32), 1791), ((255, 255, 255, 255), 1671), ((108, 101, 100, 32), 1522), ((100, 32, 116, 111), 1519), ((97, 105, 108, 101), 1513), ((105, 108, 101, 100), 1513), ((70, 97, 105, 108), 1505), ((101, 100, 32, 116), 1503)]
